In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
dataset_dir = "kaggle/input/garbage-classification-6-classes-775class"

class_labels = os.listdir(dataset_dir)

In [ ]:
class_labels

In [ ]:
num_samples_per_class = 3
plt.figure(figsize=(15, 10))
for i, label in enumerate(class_labels, start=1):
    # image filenames for the current class
    image_files = os.listdir(os.path.join(dataset_dir, label))

    sample_images = random.sample(image_files, num_samples_per_class)

    for j, image_file in enumerate(sample_images, start=1):

        plt.subplot(len(class_labels), num_samples_per_class, (i-1)*num_samples_per_class + j)
        img = mpimg.imread(os.path.join(dataset_dir, label, image_file))
        plt.imshow(img)
        
        plt.title(f"Class: {label}")
        plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
class_distribution = {label: len(os.listdir(os.path.join(dataset_dir, label))) for label in class_labels}

In [ ]:
plt.figure(figsize=(8, 6))
plt.bar(class_distribution.keys(), class_distribution.values())
plt.xlabel('Class')

plt.ylabel('Number of Images')
plt.title('Class Distribution')

plt.xticks(rotation=45)
plt.show()

In [ ]:
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input

In [ ]:
def preprocess_image(image_path, target_size=(224, 224)):


    img = image.load_img(image_path, target_size=target_size)

    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)

    img_preprocessed = preprocess_input(img_batch)


    return img_preprocessed

In [ ]:


def plot_images(original, preprocessed):
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    axs[0].imshow(original)
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    preprocessed_image = np.squeeze(preprocessed, axis=0)
    axs[1].imshow(preprocessed_image)
    axs[1].set_title('Preprocessed Image')
    axs[1].axis('off')

    plt.show()

image_path = "kaggle/input/garbage-classification-6-classes-775class/battery/battery102.jpg"
preprocessed_image = preprocess_image(image_path)
original_image = cv2.imread(image_path)
plot_images(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB), preprocessed_image)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
X = []
y = []
for label_index, label in enumerate(class_labels):
    label_dir = os.path.join(dataset_dir, label)

    for image_file in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_file)

        image = load_img(image_path, target_size=(224, 224))
        image = img_to_array(image)
        
        image = np.expand_dims(image, axis=0)
        image = image / 255.0 
         # Normalize
        X.append(image)
        y.append(label_index)

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
y = to_categorical(y, num_classes=len(class_labels))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, ResNet152, MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
num_classes=6

In [ ]:
X_train = X_train.reshape(-1, 224, 224, 3)
X_test = X_test.reshape(-1, 224, 224, 3)


In [ ]:
def evaluate_model(model, name, X_train, y_train, X_test, y_test):
    print(f"Evaluating {name}...")
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))
    
    y_pred = np.argmax(model.predict(X_test), axis=1)

    accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)

    print(f"Accuracy of {name}: {accuracy}")
    print(classification_report(np.argmax(y_test, axis=1), y_pred))



In [ ]:
# VGG16
base_model = VGG16(weights='imagenet', include_top=False)
x = base_model.output

x = GlobalAveragePooling2D()(x)

predictions = Dense(6, activation='softmax')(x)

vgg16_model = Model(inputs=base_model.input, outputs=predictions)


evaluate_model(vgg16_model, "VGG16", X_train, y_train, X_test, y_test)

In [ ]:
base_model = VGG19(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)

predictions = Dense(num_classes, activation='softmax')(x)

vgg19_model = Model(inputs=base_model.input, outputs=predictions)
evaluate_model(vgg19_model, "VGG19", X_train, y_train, X_test, y_test)

In [ ]:
# ResNet50
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output

x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

resnet50_model = Model(inputs=base_model.input, outputs=predictions)
evaluate_model(resnet50_model, "ResNet50", X_train, y_train, X_test, y_test)

In [ ]:
base_model = ResNet152(weights='imagenet', include_top=False)
x = base_model.output

x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

resnet152_model = Model(inputs=base_model.input, outputs=predictions)
evaluate_model(resnet152_model, "ResNet152", X_train, y_train, X_test, y_test)

In [ ]:
# MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)

predictions = Dense(num_classes, activation='softmax')(x)
mobilenetv2_model = Model(inputs=base_model.input, outputs=predictions)
evaluate_model(mobilenetv2_model, "MobileNetV2", X_train, y_train, X_test, y_test)

In [ ]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
def create_vgg19_model(input_shape, num_classes):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output

    x = GlobalAveragePooling2D()(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


def create_resnet152_model(input_shape, num_classes):
    base_model = ResNet152(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output

    x = GlobalAveragePooling2D()(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [ ]:

vgg19_weight = 0.6
resnet152_weight = 0.4

#Final Model=Weighted Ensemble Model of VGG19 and RESNET152

In [ ]:

vgg19_model = create_vgg19_model(input_shape, num_classes)

vgg19_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

vgg19_model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))


resnet152_model = create_resnet152_model(input_shape, num_classes)
resnet152_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

resnet152_model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))


vgg19_predictions = vgg19_model.predict(X_test)
resnet152_predictions = resnet152_model.predict(X_test)

#weighted average
weighted_predictions = (vgg19_weight * vgg19_predictions) + (resnet152_weight * resnet152_predictions)

#accuracy of the weighted ensemble
weighted_ensemble_accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(weighted_predictions, axis=1))
#print("Weighted Ensemble Accuracy:", weighted_ensemble_accuracy)


In [ ]:
def weighted_ensemble(models, weights):

    ensemble_inputs = [model.input for model in models]

    ensemble_outputs = [model.output for model in models]
    weighted_average = Dense(6, activation='softmax', name='weighted_average')(ensemble_outputs)

    weighted_model = Model(inputs=ensemble_inputs, outputs=weighted_average)

    weighted_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return weighted_model


In [ ]:
weighted_ensemble_model = weighted_ensemble([vgg19_model, resnet152_model], [0.6, 0.4])  


weighted_ensemble_model.save("abc.h5")